In [ ]:
pip install transformers datasets torch scikit-learn pandas langdetect

In [ ]:
pip install datasets

In [ ]:
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from transformers import Trainer, TrainingArguments
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import re

In [ ]:
import pandas as pd

# Load the dataset
df = pd.read_csv('/content/SupportDataset.csv')

# Check the first few rows
print(df.head())
print(df.columns)

                                             subject  \
0                           Customer Support Inquiry   
1                      Data Analytics for Investment   
2                                           Security   
3  Concerns About Securing Medical Data on 2-in-1...   
4                           Problem with Integration   

                                                body  \
0  Seeking information on digital strategies that...   
1  I am contacting you to request information on ...   
2  Dear Customer Support, I am reaching out to in...   
3  Inquiring about best practices for securing me...   
4  The integration stopped working unexpectedly, ...   

                                              answer     type  \
0  We offer a variety of digital strategies and s...  Request   
1  I am here to assist you with data analytics to...  Request   
2  Dear [name], we take the security of medical d...  Request   
3  Thank you for your concern regarding securing ...  Request   
4

In [ ]:
df = df.drop(columns=['tag_3', 'tag_4', 'tag_5', 'tag_6', 'tag_7', 'tag_8'])

df = df.dropna(subset=['subject', 'body', 'answer'])

df['text'] = df['subject'] + " " + df['body']



In [ ]:
print(df['queue'].unique())


['Customer Service' 'Technical Support' 'IT Support' 'Product Support'
 'Billing and Payments' 'Service Outages and Maintenance'
 'Human Resources' 'Returns and Exchanges' 'Sales and Pre-Sales'
 'General Inquiry']


In [ ]:
label_map = {label: idx for idx, label in enumerate(sorted(df['queue'].unique()))}

df['queue'] = df['queue'].map(label_map)

print("Label Mapping:", label_map)
print(df[['queue']].head())


Label Mapping: {'Billing and Payments': 0, 'Customer Service': 1, 'General Inquiry': 2, 'Human Resources': 3, 'IT Support': 4, 'Product Support': 5, 'Returns and Exchanges': 6, 'Sales and Pre-Sales': 7, 'Service Outages and Maintenance': 8, 'Technical Support': 9}
   queue
0      1
1      1
2      1
3      9
4      4


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, DatasetDict
import torch
from sklearn.model_selection import train_test_split

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Assuming your original DataFrame is df
# Convert DataFrame to Hugging Face Dataset (this is required to use `map`)
dataset = Dataset.from_pandas(df)

# Split dataset using scikit-learn (train_test_split)
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
if '__index_level_0__' in train_df.columns:
    train_df = train_df.drop(columns=['__index_level_0__'])

if '__index_level_0__' in test_df.columns:
    test_df = test_df.drop(columns=['__index_level_0__'])

# Convert the DataFrames back to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize the train and test datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

num_labels = len(label_map)

# Map string labels to integers
label_map = {label: idx for idx, label in enumerate(unique_labels)}

train_dataset = train_dataset.map(lambda x: {'labels': label_map[x['queue']]})
test_dataset = test_dataset.map(lambda x: {'labels': label_map[x['queue']]})


# Ensure the format is correct (torch tensors)
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Load the pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Define the Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,  # Tokenizer to handle batching correctly
)

# Start the training process
trainer.train()

# After training, evaluate the model
results = trainer.evaluate()

# Print results
print(results)

Map:   0%|          | 0/8710 [00:00<?, ? examples/s]

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Map:   0%|          | 0/8710 [00:00<?, ? examples/s]

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-7-621989787bc9>:64: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Epoch,Training Loss,Validation Loss
1,1.635600,1.612886
2,1.494300,1.551557
3,1.309800,1.543035


{'eval_loss': 1.5430349111557007, 'eval_runtime': 14.985, 'eval_samples_per_second': 145.346, 'eval_steps_per_second': 18.218, 'epoch': 3.0}


In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
from sklearn.model_selection import train_test_split

# Tokenizer for BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenization function
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True, max_length=128)

# Convert DataFrame to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Split dataset using scikit-learn
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Remove any unwanted index column
train_df = train_df.drop(columns=['__index_level_0__'], errors="ignore")
test_df = test_df.drop(columns=['__index_level_0__'], errors="ignore")

# Convert DataFrames to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
test_dataset = Dataset.from_pandas(test_df)

# Tokenize datasets
train_dataset = train_dataset.map(tokenize_function, batched=True)
test_dataset = test_dataset.map(tokenize_function, batched=True)

# Ensure label mapping is consistent
num_labels = len(label_map)  # Fixes issue with incorrect label count

# Rename 'queue' to 'labels' for classification
train_dataset = train_dataset.rename_column("queue", "labels")
test_dataset = test_dataset.rename_column("queue", "labels")

# Define required columns for Hugging Face Trainer
required_columns = ["input_ids", "attention_mask", "labels"]

# Identify extra columns
extra_columns = [col for col in train_dataset.column_names if col not in required_columns]

# Remove extra columns dynamically
train_dataset = train_dataset.remove_columns(extra_columns)
test_dataset = test_dataset.remove_columns(extra_columns)

print("Remaining columns in train dataset:", train_dataset.column_names)

# Convert datasets into correct tensor format
train_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
test_dataset.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Load the pre-trained BERT model for classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_labels)

# Define Training Arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,  # Reduce if overfitting
    weight_decay=0.01,
    logging_dir='./logs',
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

# Start Training
trainer.train()

# Evaluate Model
results = trainer.evaluate()

# Print Evaluation Results
print(results)


Map:   0%|          | 0/8710 [00:00<?, ? examples/s]

Map:   0%|          | 0/2178 [00:00<?, ? examples/s]

Remaining columns in train dataset: ['labels', 'input_ids', 'attention_mask']


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-8-d6a31d063abc>:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wand

Epoch,Training Loss,Validation Loss
1,1.651400,1.625282
2,1.510000,1.565760


{'eval_loss': 1.5657603740692139, 'eval_runtime': 14.7803, 'eval_samples_per_second': 147.359, 'eval_steps_per_second': 18.471, 'epoch': 2.0}


In [ ]:
# Save the trained model
model.save_pretrained("./my_model")

# Save the tokenizer
tokenizer.save_pretrained("./my_tokenizer")

('./my_tokenizer/tokenizer_config.json',
 './my_tokenizer/special_tokens_map.json',
 './my_tokenizer/vocab.txt',
 './my_tokenizer/added_tokens.json')

In [ ]:
training_args.save("./training_args")

AttributeError: 'TrainingArguments' object has no attribute 'save'

In [ ]:
# Evaluate on the test set
test_results = trainer.evaluate(test_dataset)
print(test_results)

{'eval_loss': 1.5430349111557007, 'eval_runtime': 14.0797, 'eval_samples_per_second': 154.69, 'eval_steps_per_second': 19.39, 'epoch': 3.0}


In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification

# Load the saved model and tokenizer
offline_model = BertForSequenceClassification.from_pretrained("./my_model")
offline_tokenizer = BertTokenizer.from_pretrained("./my_tokenizer")

print("Model loaded successfully!")


Model loaded successfully!


In [ ]:
import torch

def predict(text):
    # Tokenize input text
    inputs = offline_tokenizer(text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)

    # Run inference
    with torch.no_grad():
        outputs = offline_model(**inputs)

    # Get predicted class
    predicted_class = torch.argmax(outputs.logits, axis=-1).item()

    return predicted_class

# Example usage
text = "My payment is not processing. What should I do?"
predicted_label = predict(text)
print(f"Predicted Class: {predicted_label}")

# Reverse the mapping
reverse_label_map = {v: k for k, v in label_map.items()}  # Flip key-value pairs

# Get predicted class name
predicted_class = reverse_label_map[0]  # Using 8 from tensor([8])
print("Predicted Category:", predicted_class)

Predicted Class: 0
Predicted Category: Billing and Payments
